# EXAMPLE

In [ ]:
import pandas as pd
import rdkit.Chem.PandasTools as pt
from matplotlib.pyplot import plot as plt
import seaborn as sns
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
from usrcat.toolkits.rd import generate_moments
from usrcat.sim import *
from rdkit.Chem import PandasTools as pt
import pandas as pd
import numpy as np
import sys

In [ ]:
ltkb = pd.read_csv("../lktb_inchis.csv")
print(ltkb.shape)
ltkb.head()

In [ ]:
# Map severity classes to strings

ltkb["SEVERITY_CLASS"] = ltkb["SEVERITY_CLASS"].map( {0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5: '5', 6 :'6', 7:'7', 8:'8'})


In [ ]:
# Split ATCs to subcodes

ltkb["ATC_CODE"] = ltkb["ATC_CODE"].fillna("-")
ATCs = ltkb["ATC_CODE"]

atc_1 = []
atc_2 = []
atc_3 = []
atc_4 = []
atc_5 = []


for el in ATCs.tolist():
    try:
        el = el.split(" ")[0]
        if len(el) == 7:
            atc_1.append(el[0])
            atc_2.append(el[0:3])
            atc_3.append(el[0:4])
            atc_4.append(el[0:5])
            atc_5.append(el[0:7])
        else:
            atc_1.append("-")
            atc_2.append("-")
            atc_3.append("-")
            atc_4.append("-")
            atc_5.append("-")            

    except:
        print (el)
        atc_1.append("-")
        atc_2.append("-")
        atc_3.append("-")
        atc_4.append("-")
        atc_5.append("-")

ltkb["ATC_1"] = atc_1
ltkb["ATC_2"] = atc_2
ltkb["ATC_3"] = atc_3
ltkb["ATC_4"] = atc_4
ltkb["ATC_5"] = atc_5

In [ ]:
ltkb.shape

In [ ]:
sns.countplot(x=ltkb["ATC_2"])

In [ ]:
# Let's get the tag availability for the database

n_entries = []
for col in ltkb.columns:
    n_entries.append(ltkb[col].describe()["count"])
    
n_entries = np.asarray(n_entries).reshape(1, -1)
n_entries = pd.DataFrame(n_entries, columns=ltkb.columns.tolist())
n_entries

In [ ]:
# Let's remove compounds with no InChI

ltkb = ltkb.fillna("-")
ltkb = ltkb[ltkb["InChI"] !="-"]
ltkb.shape

In [ ]:
# Countplot of different severity_class tag

sns.countplot(ltkb["SEVERITY_CLASS"])

In [ ]:
# Compute USR-CAT moments

def get_moments(mols):
    matrix = np.zeros((len(mols), 600))
    for index, mol in enumerate(mols):
        try:
            vector = np.ravel(generate_moments(mol))
            matrix[index, :] = vector
        except:
            sys.exit(-1)
    return matrix

In [ ]:
# Get mols from InChIs

ltkb["ROMol"] = [AllChem.MolFromInchi(i) for i in ltkb["InChI"]]

In [ ]:
# Get conformations for compounds
errors = []
for index, row in ltkb.iterrows():
    try:
        AllChem.EmbedMultipleConfs(row["ROMol"], numConfs=10)
    except:
        print ("error at index ", index)
        errors.append(index)
        
        

In [ ]:
# Get subframe with no erroneus compounds at computing conformers 

ltkb_1 = ltkb[~ltkb.index.isin(errors)].reset_index(drop=True)
erroneus = []
for index, row in ltkb_1.iterrows():
    if len(row["ROMol"].GetConformers()) != 10:
        erroneus.append(index)
ltkb_1 = ltkb_1[~ltkb_1.index.isin(erroneus)].reset_index(drop=True)

In [ ]:
# Get USR-CAT moments

matrix = get_moments(ltkb_1["ROMol"].tolist())
dili = pd.DataFrame(matrix)

In [ ]:
# Now let's cluster ltkb compounds by their 3D-shape/pharmacophoric properties

from sklearn.cluster import KMeans

km = KMeans(n_clusters=30, random_state=46)
km.fit(matrix)
ltkb_1["cluster"] = km.labels_
mols = ltkb_1["ROMol"].tolist()

# Save the file to open it again with PandasTool (to see molecule depiction)
pt.WriteSDF(ltkb_1, "dili_cluster3d.sdf", properties=ltkb_1.columns)
ltkb_1 = pt.LoadSDF("dili_cluster3d.sdf")

In [ ]:
# Convert the cluster column to integer

ltkb_1["cluster"] = ltkb_1["cluster"].astype(int)

In [ ]:
ltkb_bk = ltkb_1.copy()
new = ltkb_bk.copy()

In [ ]:
new["cluster2"] = ""
for cluster in np.unique(new["cluster"]):
    subset = new[new["cluster"] == cluster]
    morgans = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, useFeatures=False) for mol in subset["ROMol"]]
    if len(morgans) > 10:
        #n_clu = int(len(morgans) * 0.2)
        n_clu = 5
        km = KMeans(n_clusters=n_clu, random_state=46)
        km.fit(morgans)
        subset["cluster2"] = km.labels_
        for index, row in subset.iterrows():
            new.at[index, "cluster2"] = row["cluster2"]
        
    else:
        continue
    

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
len(np.unique(new["ATC_2"]))

In [ ]:
new = new.sort_values(by=["cluster", "cluster2"])

In [ ]:
 # ["SEVERITY_CLASS", "ATC_CODE"]
    
    
most = new[new["VERIFIED_DILI_CONCERN"] == "vMost-DILI-Concern"]
no = new[new["VERIFIED_DILI_CONCERN"] == "vNo-DILI-Concern"]

grid = sns.FacetGrid(new[["cluster", "cluster2","VERIFIED_DILI_CONCERN", "ATC_1"]], 
                     col="cluster2", row="cluster",
                     size=3, aspect=1, legend_out=False, sharex=False)


# Draw a horizontal line to show the starting point

# Draw a line plot to show the trajectory of each random walk
fig = grid.map(sns.countplot, "ATC_1", palette="Blues",
               order=np.unique(new["ATC_1"]))

fig.set_xticklabels(rotation=90)
# Adjust the tick positions and labels
# fig.set(xticks=np.arange(9))
#          #xlim=(-0.5, 9.5) )

fig.add_legend()
# Adjust the arrangement of the plots
fig.savefig("test.png",dpi=50)

In [ ]:
new.sort_values(by=["cluster", "cluster2"])[["ROMol", "cluster", "cluster2","ATC_CODE", "SEVERITY_CLASS"]]

In [ ]:
new2 = new.copy(deep=True)

In [ ]:
new2["SEVERITY_CLASS"] = new2["SEVERITY_CLASS"].replace("-", 0).astype(int)
new2["SEVERITY_CLASS"] = new2["SEVERITY_CLASS"].replace(0, np.nan)

In [ ]:
grouped = new2.groupby(by=["cluster","cluster2", "ATC_1"])["SEVERITY_CLASS"].apply(list).reset_index()

In [ ]:
grouped